In [ ]:
# Установка необходимых библиотек

# для эмбеддингов
!pip install navec
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
# для парсинговых правил
!pip install yargy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-08-31 10:14:05--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.3s    

2022-08-31 10:14:09 (15.3 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 75 kB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 31.5 MB/s

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-08-31 10:14:24--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.4s    

2022-08-31 10:14:28 (14.9 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’ saved [53012480/53012480]



In [ ]:
def preprocess_text(text):
    tokens = text.split()
    tokens = [token.lower() for token in tokens if token.lower() not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [ ]:
# импорт библиотек

# открыть данные
import pandas as pd 

# предобработка данных
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

# для поиска похожих слов
from navec import Navec
import torch
import torch.nn as nn

# парсировщики
from yargy import Parser, rule, or_
from yargy.predicates import gram, dictionary

# засекал время обработки данных
from datetime import datetime
import time

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/bewise.ai/test_data.csv')

In [ ]:
data[data['dlg_id']==4][:30]

,dlg_id,line_n,role,text
302,4,0,client,Алло
303,4,1,client,Алло
304,4,2,client,Добрый
305,4,3,manager,Вот по виду платежи пообщаться помните мы вот ...
306,4,4,client,Угу
307,4,5,client,Ну в принципе да
308,4,6,manager,Я понял ну мы хотели бы просто предложить тако...
309,4,7,client,Угу
310,4,8,manager,Готовы будем для вашего клиента это использова...
311,4,9,manager,Наверное в рамках недели полторы наверное займ...


In [ ]:
# data['greeting'] = [0 for k in range(data.shape[0])]
# data['intro'] = [0 for k in range(data.shape[0])]
# data['name'] = [0 for k in range(data.shape[0])]
# data['company_name'] = [0 for k in range(data.shape[0])]
# data['goodbye'] = [0 for k in range(data.shape[0])]
# data['final'] = [0 for k in range(data.shape[0])]

In [ ]:
nltk.download("stopwords")
#--------#


#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
russian_stopwords.append(['угу', 'ага'])

In [ ]:
data['new_text'] = data['text'].apply(lambda x: preprocess_text(x))

In [ ]:
# data[data['role'] ==  'manager'][:30]

In [ ]:
# def preprocess_text_1(text):
#     tokens = text.split()
#     # tokens = [token.lower() for token in tokens if token.lower() not in russian_stopwords\
#     #           and token != " " \
#     #           and token.strip() not in punctuation]
#     tokens = [token.lower() for token in tokens]
#     text = " ".join(tokens)
    
#     return text

In [ ]:
# def greeting(text):
#     perem = 0
#     names = []

#     emb = NewsEmbedding()
#     ner_tagger = NewsNERTagger(emb)

#     doc = Doc(text)
#     doc.segment(segmenter)

#     doc.tag_ner(ner_tagger)

#     for span in doc.spans:
#         if span.type == PER:
#             perem = 1
#             names.append(span.text)

#     return ' '.join(names)

In [ ]:
# data['name'] = data[data['role'] == 'manager']['text'].apply(lambda x: greeting(x))

In [ ]:
# !pip install natasha

In [ ]:
# from natasha import (
#     Segmenter,
#     MorphVocab,
    
#     NewsEmbedding,
#     NewsMorphTagger,
#     NewsSyntaxParser,
#     NewsNERTagger,
    
#     PER,
#     NamesExtractor,

#     Doc
# )
# morph_vocab = MorphVocab()

In [ ]:
# emb = NewsEmbedding()
# segmenter = Segmenter()
# morph_tagger = NewsMorphTagger(emb)


# text = 'Меня зовут ангелина компания диджитал бизнес'
# doc = Doc(text)

In [ ]:
# names_extractor = NamesExtractor(morph_vocab)

In [ ]:
# doc.segment(segmenter)

In [ ]:
# doc.tag_morph(morph_tagger)

In [ ]:
# # for word in doc.sents[0].morph.tokens:
#     if word.pos == 'PROPN':
#         print(word.pos)

# Navec

In [ ]:
path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
cos = nn.CosineSimilarity()

In [ ]:
# v1 = navec['инсаф']
# v2 = navec['гор']

# cos(torch.tensor([v1]), torch.tensor([v2])).item()

In [ ]:
# v1 = navec['до'] + navec['свидания']
# v2 = navec['прощай']

# cos(torch.tensor([v1]), torch.tensor([v2])).item()

In [ ]:
# dictt = {dlg_id : [greeting, with_name, name, company_name, with_goodbye, final]}

In [ ]:
RULE_FOR_NAME = or_(rule(dictionary({'меня', 'это', 'я'}),
                 gram('Name'),),
          rule(dictionary({'зовут'}),
               gram('Name')))

PARSER_FOR_NAME = Parser(RULE_FOR_NAME)

In [ ]:
def simular_greating(text):
      global info_dict
      if text.dlg_id not in info_dict:
            info_dict[text.dlg_id] = [[],[],[],[],[],0]

      
      perem = 0
      for word in text.new_text.split():
          # print(word))
          if word in navec:
              if cos(torch.tensor([navec[word]]), torch.tensor([navec['привет']])).item() > 0.63:
                  info_dict[text.dlg_id][0].append(text.text)
                  break
              elif cos(torch.tensor([navec[word]]), torch.tensor([navec['здравствуйте']])).item() > 0.52:
                  info_dict[text.dlg_id][0].append(text.text)
                  break

def simular_name(text):
      global info_dict
      name = 0

      for word in text.new_text.split():
          # print(word))
          wordd = ''
          if word in navec:
              if cos(torch.tensor([navec[word]]), torch.tensor([navec['дмитрий']])).item() > 0.5:
                  for match in PARSER_FOR_NAME.findall(text.text):
                        wordd = [x.value for x in match.tokens][-1]
                  if len(wordd) > 1:           
                      info_dict[text.dlg_id][1].append(text.text)
                      info_dict[text.dlg_id][2].append(wordd)
              elif cos(torch.tensor([navec[word]]), torch.tensor([navec['анастасия']])).item() > 0.5:
                  for match in PARSER_FOR_NAME.findall(text.text):
                        wordd = [x.value for x in match.tokens][-1]
                  if len(wordd) > 1:           
                      info_dict[text.dlg_id][1].append(text.text)
                      info_dict[text.dlg_id][2].append(wordd)


def company_name(text):
      global info_dict

      sentence = text.new_text.split()
      company_name = ''
      for k in range(len(sentence)):
          # print(word))
          if sentence[k] in navec:
              if cos(torch.tensor([navec[sentence[k]]]), torch.tensor([navec['название'] + navec['компании']])).item() > 0.5:
                  perem = 1
                  if len(sentence) - 3 > k:
                        for _ in range(1,4):
                            if sentence[k + _] in navec:
                                if cos(torch.tensor([navec[sentence[k + _]]]), torch.tensor([navec['компания']])).item() > 0.4:
                                      company_name = company_name + ' ' + sentence[k+_]
                            else:
                                company_name = company_name + ' ' + sentence[k+_]
                  elif len(sentence) - 2 > k:
                      for _ in range(1,3):
                            if sentence[k + _] in navec:
                                if cos(torch.tensor([navec[sentence[k + _]]]), torch.tensor([navec['компания']])).item() > 0.4:
                                      company_name = company_name + ' ' + sentence[k+_]
                            else:
                                company_name = company_name + ' ' + sentence[k + _]
                  if len(company_name)>1:
                      info_dict[text.dlg_id][3].append(company_name[1:])
                  break


def simular_goodbye(text):
      global info_dict
      perem = 0
      for word in text.new_text.split():
          # print(word))
          if word in navec:
              if cos(torch.tensor([navec[word]]), torch.tensor([navec['до'] + navec['свидания']])).item() > 0.63 and word != 'до':
                  # print(word)
                  if len(info_dict[text.dlg_id][0]) > 0:
                        info_dict[text.dlg_id][5] = 1
                  info_dict[text.dlg_id][4].append(text.text)
                  perem = 1
                  break
              elif cos(torch.tensor([navec[word]]), torch.tensor([navec['доброго'] + navec['вечера']])).item() > 0.55:
                  if len(info_dict[text.dlg_id][0]) > 0:
                        info_dict[text.dlg_id][5] = 1
                  info_dict[text.dlg_id][4].append(text.text)
                  # print(word)
                  perem = 1
                  break

In [ ]:
info_dict = {}
start_time = datetime.now()

data[data['role'] == 'manager'].apply(lambda x: simular_greating(x), axis=1)
data[data['role'] == 'manager'].apply(lambda x: simular_name(x), axis=1)
data[data['role'] == 'manager'].apply(lambda x: company_name(x), axis=1)
data[data['role'] == 'manager'].apply(lambda x: simular_goodbye(x),axis=1)

print(datetime.now() - start_time)

0:00:01.290840


In [ ]:
# (1000000/500) /60

33.333333333333336

In [ ]:
for k in info_dict:
    for line in info_dict[k]:
        if len(line) == 0:
            print("None")
        else:
            print(line)
    print()

['Алло здравствуйте']
['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается']
['ангелина']
['диджитал бизнес']
['Всего хорошего до свидания']
1

['Алло здравствуйте']
['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается']
['ангелина']
['диджитал бизнес']
['Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего', 'До свидания']
1

['Алло здравствуйте']
['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там']
['ангелина']
['диджитал бизнес']
[]
0

['Алло дмитрий добрый день', 'Добрый меня максим зовут компания китобизнес удобно говорить']
['Добрый меня максим зовут компания китобизнес удобно говорить']
['максим']
['китобизнес']
['Угу все хорошо да понедельника тогда всего доброго']
1

[]
[]
[]
[]
['Во вторник все ну с вами да

# spacy

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-29 19:49:59.500887: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
import spacy

In [ ]:
nlp = spacy.load("ru_core_news_lg")

In [ ]:
doc1 = nlp("8зво9нит5те6сю7фв83с")
print([(w.text, w.pos_) for w in doc1])

[('8зво9нит5те6сю7фв83с', 'NUM')]


In [ ]:
doc2 = nlp("@alesasner @asfdfgasd")
print([(w.text, w.pos_) for w in doc2])

[('@alesasner', 'X'), ('@asfdfgasd', 'X')]


In [ ]:
doc1 = nlp("Меня зовут дмитрий")
doc2 = nlp("Алло дмитрий")
# тут высокое должно быть
print(doc1.similarity(doc2))

0.7712918248434255


In [ ]:
doc1 = nlp("Меня зовут дмитрий, компания")
doc2 = nlp("Алло дмитрий добрый день")
# низкое
print(doc1.similarity(doc2))

0.6057535185208839


In [ ]:
doc1 = nlp("Вот смотрите вячеслав сейчас со мной я вам вышлю счет мы с вами продлили лицензию далее вы мне дадите номер не знаю")
doc2 = nlp("Меня зовут дмитрий, компания")
# низкое
print(doc1.similarity(doc2))

0.5397332018250637


In [ ]:
doc1 = nlp("это анастасия")
doc2 = nlp("Меня зовут дмитрий, компания")
# высокое
print(doc1.similarity(doc2))

0.5540353129752361


In [ ]:
import re

In [ ]:
String = "я николай компания рога и копыта чем могу помочь"
stringg = 'рога и копыта'
print(re.search(stringg, String).start())
print(re.search(stringg, String).end())

19
32


In [ ]:
from spacy.training import Example

In [ ]:
import spacy
import random

TRAINING_DATA = [
    ("Добрый меня максим зовут компания китобизнес удобно говорить", 
    {"entities": [(34,44,"COMPANY"),
                  (13,18, "PERSON")]}),
    ("Вот смотрите вячеслав сейчас со мной я вам вышлю счет мы с вами продлили лицензию далее вы мне дадите номер не знаю", 
    {"entities": []}),
    ("Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается", 
    {"entities": [(19,44,"COMPANY"),
                  (11,19, "PERSON")]}),
                 
    ("зовут меня игорь хочу представить вам средство от комаров", 
    {"entities": [(11,16, "PERSON")]}),

    ("Угу все хорошо да понедельника тогда всего доброго", 
    {"entities": []}),

    ("зовут меня игорь хочу представить вам средство от комаров", 
    {"entities": [(11,16, "PERSON")]}),
    ("алевтина здравствуйте меня зовут инсаф компания бизнесс солюшн мы представляем юридическую помощь", 
    {"entities": [(33,38, "PERSON"),
                  (48,62,"COMPANY")]}),
    ("я николай компания рога и копыта чем могу помочь", 
    {"entities": [(2,9, "PERSON"),
                  (19,32,"COMPANY")]}),
    ("Так дмитрий", 
    {"entities": []})
]



for i in range(5):
    random.shuffle(TRAINING_DATA)
    for batch in spacy.util.minibatch(TRAINING_DATA,size=1):
        for text, annotations in batch:
          example = Example.from_dict(nlp.make_doc(text), annotations)
          nlp.update([example])
        # texts = [text for text, annotation in batch]
        # annotations = [annotation for text, annotation in batch]
        # nlp.update(texts, annotations)
        
nlp.to_disk("model")

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Меня зовут ангелина компания диджитал бизнес звони..." with entities "[(19, 44, 'COMPANY'), (11, 19, 'PERSON')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Добрый меня максим зовут компания китобизнес удобн..." with entities "[(34, 44, 'COMPANY'), (13, 18, 'PERSON')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


# Yargy

In [ ]:

# text = 'Добрый меня максим зовут компания китобизнес удобно говорить'
# text = 'Добрый день дмитрий меня зовут инсаф компания китобизнес удобно говорить'
text = 'Я гор'
# text = 'Вот смотрите вячеслав сейчас со мной я вам вышлю счет мы с вами продлили лицензию далее вы мне дадите номер не знаю'
# text = "Так дмитрий"
# text = 'Алло дмитрий'

for match in PARSER_FOR_NAME.findall(text):
    print ([x.value for x in match.tokens][-1])

In [ ]:
from yargy.tokenizer import MorphTokenizer

text = 'Компания рога и копыта звоню вам'
text = 'Компания дижитал текноложис звоню вам'

TOKENIZER = MorphTokenizer()
list(TOKENIZER(text))

[MorphToken(
     value='Компания',
     span=[0, 8),
     type='RU',
     forms=[Form('компания', Grams(NOUN,femn,inan,nomn,sing))]
 ), MorphToken(
     value='дижитал',
     span=[9, 16),
     type='RU',
     forms=[Form('дижитал', Grams(ADJF,Abbr,Fixd,masc,nomn,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,gent,masc,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,datv,masc,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,accs,masc,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,ablt,masc,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,loct,masc,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,femn,nomn,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,femn,gent,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,datv,femn,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,accs,femn,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,ablt,femn,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,femn,loct,sing)),
      Form('дижитал', Grams(ADJF,Abbr,Fixd,neut,nomn,sin

In [ ]:
RULE_FOR_COMPANY = or_(rule(dictionary({'компания'}),
                 gram('Name'),),
          rule(dictionary({'зовут'}),
               gram('Name')))


PARSER_FOR_COMPANY = Parser(RULE_FOR_COMPANY)


# Предварительные результаты и проблемы  


С помощью эмбеддингов слов и модуля yargy удалось выполнить задачу. Пробовал с помощью yargy парсить название компаний, однако не придумал, как решать проблему с компаниями в названии которых есть глаголы, наречия и другие, возможно неизвестные токены (не смог решить проблему остановки). Данная реализация проблемна только с компаниями, название которых больше трех.  

С именами тоже проблема: если встретится просто имя, без предварительной фразы "зовут" или местоимений я, это, меня, то имя просто пропустится. Если убрать эти условия, то имена будут доставаться все. Для полноты это хорошо, но для точности - плохо. Тут можно варьировать взависимости от задачи.  

Приветствие и прощание вытаскивается безошибочно (на моих тестах).


Пробовал различные ner библиотеки (natasha, deeppavlov, stanza-ru) они все показали плохое качество (предположительно из-за того, что все тексты в нижнем регистре). В целом, можно попробовать было сначала решить задачу проставления больших букв и пунктуации, но данная задача, на мой взгляд, сводится именно к ner'у, поэтому я так делать не стал.